## Scheduling API Notebook

[Guide can be found here](https://docs.google.com/document/d/1ZIPTIQklAOKgu7H3-7WIz6DmV6kbmSB3yVs-98hbP8I/edit?resourcekey=0-7mdbvb1o-2v7gUDlx1LD5Q)

### Prepare a pipeline

In [1]:
from kfp.v2.dsl import component


@component
def test(name: str) -> str:
    return f"Hello {name}"

In [37]:
from kfp.v2.dsl import pipeline


@pipeline(name='test')
def pipeline():
    test_task = test("Matilda")

### Compile pipeline

In [39]:
from kfp.v2 import compiler
import warnings
warnings.filterwarnings('ignore')


compiler.Compiler().compile(pipeline_func=pipeline,
                            package_path='test.json')

In [40]:
pipeline_root = 'gs://pipeline-scheduling-test-feb-23'

#### (OPTIONAL) Run only one time to create the pipeline root bucket

In [5]:
#!gsutil mb $pipeline_root

In [6]:
## Run the pipeline to test

PROJECT_ID = 'wortz-project-352116'
LOCATION = 'us-central1'
PROJECT_NUM = 679926387543

from google.cloud import aiplatform

job = aiplatform.PipelineJob(display_name = f'test-non-scheduled',
                             template_path = 'test.json',
                             pipeline_root = pipeline_root,
                             # parameter_values = PIPELINE_PARAMETERS,
                             project = PROJECT_ID,
                             location = LOCATION,
                             enable_caching=False)
job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/679926387543/locations/us-central1/pipelineJobs/test-20230207002658
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/679926387543/locations/us-central1/pipelineJobs/test-20230207002658')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/test-20230207002658?project=679926387543


### Create the request to schedule the pipeline

Note you can inspect the contents of the compiled pipeline `json` object. It should look like this:

```
{
  "pipelineSpec": {
    "components": {
      "comp-test": {
        ....
    "deploymentSpec": {
           ...
          }
        }
      }
    },
    "pipelineInfo": {
      "name": "test"
    },
    "root": {
      "dag": {
        "tasks": {
          "test": {
            ...
                    }
                  }
                }
              }
            },
            "taskInfo": {
              "name": "test"
            }
          }
        }
      }
    },
    "schemaVersion": "2.0.0",
    "sdkVersion": "kfp-1.8.18"
  },
  "runtimeConfig": {}
}
```

We will shape the request by taking the `PipelineSpec` object from this json and addit to the required API call like so:

```
PROJECT_ID=<Project ID for the request>
LOCATION=<Location or region of the pipeline run, for example, us-west2>

curl -i -X POST -H "Content-Type: application/json" -H "Authorization: Bearer $(gcloud auth print-access-token)" \
https://${LOCATION}-aiplatform.googleapis.com/v1beta1/projects/${PROJECT_ID}/locations/${LOCATION}/schedules \
-d '{
  "display_name":"<SCHEDULE_NAME>",
  "start_time": "<SCHEDULE_START_TIME>",
  "cron": "<CRON_EXPRESSION>",
  "max_concurrent_run_count": "<MAX_CONCURRENT_RUN_COUNT>",
  "create_pipeline_job_request": <CREATEPIPELINEJOB_API_REQUEST_TEMPLATE>
}'
```

In [43]:
import json
import requests

PROJECT_ID = 'wortz-project-352116'
LOCATION = 'us-central1'
PROJECT_NUM = 679926387543

pipeline_file = open('test.json')
pipeline_spec = json.load(pipeline_file)
pipeline_file.close()

In [44]:
pipeline_spec

{'pipelineSpec': {'components': {'comp-test': {'executorLabel': 'exec-test',
    'inputDefinitions': {'parameters': {'name': {'type': 'STRING'}}},
    'outputDefinitions': {'parameters': {'Output': {'type': 'STRING'}}}}},
  'deploymentSpec': {'executors': {'exec-test': {'container': {'args': ['--executor_input',
       '{{$}}',
       '--function_to_execute',
       'test'],
      'command': ['sh',
       '-c',
       '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet     --no-warn-script-location \'kfp==1.8.18\' && "$0" "$@"\n',
       'sh',
       '-ec',
       'program_path=$(mktemp -d)\nprintf "%s" "$0" > "$program_path/ephemeral_component.py"\npython3 -m kfp.v2.components.executor_main                         --component_module_path                         "$program_path/ephemeral_component.py"                         "$@"\n',
       '

In [45]:
### Base request for scheduling

cron_expression = "TZ=America/New_York 0 * * * *" #run every hour
display_name = "hello_world_scheduling"
pipeline_spec['runtimeConfig'].update({"gcsOutputDirectory": pipeline_root}) #add the gs bucket for artifacts


create_job_request = {
    "parent": f"projects/{PROJECT_NUM}/locations/{LOCATION}",
    "pipelineJob" : {
        "displayName": display_name,
    }
}

create_job_request["pipelineJob"].update(pipeline_spec) #add the pipeline to the job request template

schedule_request = {
    "display_name": display_name,
    "cron": cron_expression,
    "max_concurrent_run_count": "1",
    "create_pipeline_job_request": create_job_request
}


In [46]:
schedule_request

{'display_name': 'hello_world_scheduling',
 'cron': 'TZ=America/New_York 0 * * * *',
 'max_concurrent_run_count': '1',
 'create_pipeline_job_request': {'parent': 'projects/679926387543/locations/us-central1',
  'pipelineJob': {'displayName': 'hello_world_scheduling',
   'pipelineSpec': {'components': {'comp-test': {'executorLabel': 'exec-test',
      'inputDefinitions': {'parameters': {'name': {'type': 'STRING'}}},
      'outputDefinitions': {'parameters': {'Output': {'type': 'STRING'}}}}},
    'deploymentSpec': {'executors': {'exec-test': {'container': {'args': ['--executor_input',
         '{{$}}',
         '--function_to_execute',
         'test'],
        'command': ['sh',
         '-c',
         '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ensurepip || python3 -m ensurepip --user || apt-get install python3-pip\nfi\n\nPIP_DISABLE_PIP_VERSION_CHECK=1 python3 -m pip install --quiet     --no-warn-script-location \'kfp==1.8.18\' && "$0" "$@"\n',
         'sh',
         '-ec

### Submit the request

In [56]:
#auth
import google.auth
import google.auth.transport.requests
creds, project = google.auth.default()

# creds.valid is False, and creds.token is None
# Need to refresh credentials to populate those

auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)

In [58]:
url = f'https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/schedules'
headers = {"Authorization": f"Bearer {creds.token}"}

In [48]:
requests.post(url, json=schedule_request, headers=headers).json()

{'name': 'projects/679926387543/locations/us-central1/schedules/4836303049842491392',
 'displayName': 'hello_world_scheduling',
 'startTime': '2023-02-07T01:33:49.245448Z',
 'state': 'ACTIVE',
 'createTime': '2023-02-07T01:33:49.245448Z',
 'nextRunTime': '2023-02-07T02:00:00Z',
 'cron': 'TZ=America/New_York 0 * * * *',
 'maxConcurrentRunCount': '1',
 'createPipelineJobRequest': {'parent': 'projects/679926387543/locations/us-central1',
  'pipelineJob': {'displayName': 'hello_world_scheduling',
   'pipelineSpec': {'components': {'comp-test': {'executorLabel': 'exec-test',
      'inputDefinitions': {'parameters': {'name': {'type': 'STRING'}}},
      'outputDefinitions': {'parameters': {'Output': {'type': 'STRING'}}}}},
    'deploymentSpec': {'executors': {'exec-test': {'container': {'args': ['--executor_input',
         '{{$}}',
         '--function_to_execute',
         'test'],
        'command': ['sh',
         '-c',
         '\nif ! [ -x "$(command -v pip)" ]; then\n    python3 -m ens

### List out the schedules

In [59]:
url_list = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/schedules"

requests.get(url_list, headers=headers).json()

{}

### Deleting a schedule

In [55]:

DELETED_SCHEDULE_ID = 6035104975653175296

url_delete = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/schedules/{DELETED_SCHEDULE_ID}"

requests.delete(url_delete, headers=headers).json()

{'name': 'projects/679926387543/locations/us-central1/operations/8830374489091997696',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1beta1.DeleteOperationMetadata',
  'genericMetadata': {'createTime': '2023-02-07T14:56:37.875254Z',
   'updateTime': '2023-02-07T14:56:37.875254Z'}},
 'done': True,
 'response': {'@type': 'type.googleapis.com/google.protobuf.Empty'}}